# 心電図からrr間隔を出力するプログラム

In [1]:
import sys
sys.path.append('..')
from Constants import Constants

In [2]:
import glob

In [3]:
import numpy as np
import pandas as pd
import heartpy as hp
from matplotlib import pyplot as plt

In [18]:
for i, subject in enumerate(Constants.subject_name_list):
    
    df_mean = pd.DataFrame()
    
    base_PLI_FC3 = 0
    base_PLI_FC4 = 0
    base_PLI_FCz = 0

    rest_path = glob.glob(f"{Constants.path_to_matlab_repository_folder}/csv/{i}_{subject}/PLIdata/PLI_?_{subject}_rest.csv")[0]
    rest_df = pd.read_csv(rest_path)

    
    for j, phase in enumerate(Constants.phase_name_list):
        

        
        # PLI
        
        
        pathes = glob.glob(f"{Constants.path_to_matlab_repository_folder}/csv/{i}_{subject}/PLIdata/PLI_?_{subject}_{phase}.csv")
        path = pathes[0]
        
        
        save_path = f"{Constants.path_to_matlab_repository_folder}/csv/{i}_{subject}/PLIdata/mean_PLI_{subject}.csv"
        
        print("-------------------------------------------------")
        print(f"{subject}さんの{phase}フェーズ")
        df = pd.read_csv(path)

        
        columns_for_PLI = ['FpZ','FC3','FC4','FCz','O1','O2','Fp2']
        
        if phase == "rest":
            
            
            mean_values = []
            for column in columns_for_PLI:
                mean = df[column].mean()
                mean_values.append(mean)
                
                if column == "FC3":
                    base_PLI_FC3 = mean
                    
                if column == "FC4":
                    base_PLI_FC4 = mean
                    
                if column == "FCz":
                    base_PLI_FCz = mean

            # 初期化みたいな意味合い
            df_mean = pd.DataFrame([mean_values], columns=columns_for_PLI, index=[phase])

            # 2つのカラムを追加
            df_mean['mean_all'] = df_mean.mean(axis=1)
            mean_3ch = (df_mean['FC3'] + df_mean['FC4'] + df_mean['FCz']) / 3
            df_mean['mean_3ch'] = mean_3ch

            print(df_mean)
            
            
        else:
            
            mean_values = []
            concentration = []
            max_concentrations = []
            for column in columns_for_PLI:
                mean = df[column].mean()
                mean_values.append(mean)
                
                if column == "FC3":
                    concentration_FC3 = base_PLI_FC3 - mean
                    concentration.append(concentration_FC3)
                    concentrations_time_series = rest_df[[column]] - df[[column]]
                    print("-----------")
                    print(type(concentrations_time_series.max()))
                    print(concentrations_time_series.max()[column]) # これでやっとseriesじゃなくて値になる
                    max_concentrations.append(concentrations_time_series.max()[column])
                    
                if column == "FC4":
                    concentration_FC4 = base_PLI_FC4 - mean
                    concentration.append(concentration_FC4)
                    concentrations_time_series = rest_df[[column]] - df[[column]]
                    max_concentrations.append(concentrations_time_series.max()[column])
                    
                if column == "FCz":
                    concentration_FCz = base_PLI_FCz - mean
                    concentration.append(concentration_FCz)
                    concentrations_time_series = rest_df[[column]] - df[[column]]
                    max_concentrations.append(concentrations_time_series.max()[column])
            
            
            # 集中度。restからのPLI差分を計算することでいける
            print("-------------")
            print(max_concentrations)
            print(concentration)
            
            row = np.concatenate([mean_values, concentration, max_concentrations])
            column = np.concatenate([columns_for_PLI, ["concentration_FC3",
                                                       "concentration_FC4", 
                                                       "concentration_FCz", 
                                                       "max_concentration_FC3", 
                                                       "max_concentration_FC4", 
                                                       "max_concentration_FCz", 
                                                       ]])
                

            df_for_add = pd.DataFrame([row], columns=column, index=[phase])
            
            concentration_mean = (df_for_add['concentration_FC3'] + df_for_add['concentration_FC4'] + df_for_add['concentration_FCz']) / 3
            max_concentration_mean = (df_for_add['max_concentration_FC3'] + df_for_add['max_concentration_FC4'] + df_for_add['max_concentration_FCz']) / 3
            df_for_add['concentration_mean'] = concentration_mean
            df_for_add['max_concentration_mean'] = max_concentration_mean
            
            # /集中度

            
            # 2つのカラムを追加
            df_for_add['mean_all'] = df_for_add.mean(axis=1)
            mean_3ch = (df_for_add['FC3'] + df_for_add['FC4'] + df_for_add['FCz']) / 3
            df_for_add['mean_3ch'] = mean_3ch
            
            
            
            
            

            # append的な意味合い
            df_mean = pd.concat([df_mean, df_for_add])



        

    # 1subjectに対する保存処理をここのインデントで記載する
    print(df_mean)
    df_mean.to_csv(save_path)
        

-------------------------------------------------
kumakuraさんのrestフェーズ
           FpZ       FC3       FC4       FCz        O1        O2       Fp2  \
rest  0.470375  0.718243  0.530592  0.783054  0.352428  0.286579  0.390179   

      mean_all  mean_3ch  
rest  0.504493  0.677296  
-------------------------------------------------
kumakuraさんのboredomフェーズ
-----------
<class 'pandas.core.series.Series'>
0.476316318527755
-------------
[0.476316318527755, 0.418886313624333, 0.47189025908593596]
[0.27216106378406474, 0.0928625812784381, 0.22210440553168243]
-------------------------------------------------
kumakuraさんのflowフェーズ
-----------
<class 'pandas.core.series.Series'>
0.643084068775
-------------
[0.643084068775, 0.466639971518096, 0.7239726045774908]
[0.29067121357617565, 0.09518346627386182, 0.40786793047677117]
-------------------------------------------------
kumakuraさんのultraフェーズ
-----------
<class 'pandas.core.series.Series'>
0.643084068775
-------------
[0.643084068775, 0.466639971